matrix transformation only for perfect file (with path 0)

In [ ]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize

# pymatgen libraries
from pymatgen.core.structure import Structure

# import function from .py file
from get_weirdos_ver3_new_cleaned import get_dx_dz_init, get_sum_weirdos_Li_var_wo_weirdo_litype
# from get_weirdos import get_sum_weirdos_Li
# from modify_dx_dz import create_file_name, change_dx_dz
# from get_weirdos import get_dx1_dx2_dz_init, get_sum_weirdos_Li

direc = os.getcwd() # get current working directory

In [ ]:
# from get_weirdos_ver3_new_cleaned import varying_radius_vs_sumweirdosLi, plot_varying_radius_vs_sumweirdosLi

In [ ]:
# # filefile_path_ori_ref_48n24 = "./perfect_poscar/cif_matrix/ori/Li6PS5Cl_type2.cif"_path_ori_ref_48n24 = "./perfect_poscar/cif_matrix/ori/Li6PS5Cl_48n24_mod_old_2p27291_0p88814_arcA.cif"  # Replace with your file path
  # Replace with your file path
file_perfect_poscar_48n24_wo_cif = "Li6PS5Cl_optitype1n2_strictr_a_type4_weirdo1_var2" # same as: "Li6PS5Cl_type2_2iia_115_new"
file_perfect_poscar_48n24 = f"{file_perfect_poscar_48n24_wo_cif}.cif"

file_path_ori_ref_48n24 = f"./perfect_poscar/cif_matrix/ori/{file_perfect_poscar_48n24}"
file_perfect_poscar_24 = "Li6PS5Cl_24_mod_2p27291.cif"
# file_ori_ref_48n24 = "Li6PS5Cl_48n24_mod_old_2p27291_0p88814_arcA.cif"

n_file_start = 0
n_file_end = 54

activate_radius = 1
max_mapping_radius = 0.043 # 0.043 ### 0.018 # 0.015 # 0.043
max_mapping_radius_48htype2 = 0.066 # 0.076 ### 0.075 # 0.051 # 0.076
max_mapping_radius_48htype1_48htype2 = (max_mapping_radius + max_mapping_radius_48htype2) / 2

In [ ]:
if activate_radius == 1:
    file_loc_important_cols = pd.read_pickle(f'file_loc_important_cols_{max_mapping_radius}_{file_perfect_poscar_48n24_wo_cif}.pkl') 
elif activate_radius == 2:
    file_loc_important_cols = pd.read_pickle(f'file_loc_important_cols_{max_mapping_radius}_{max_mapping_radius_48htype2}_{file_perfect_poscar_48n24_wo_cif}.pkl')
elif activate_radius == 3:
    file_loc_important_cols = pd.read_pickle(f'file_loc_important_cols_{max_mapping_radius}_{max_mapping_radius_48htype2}_{max_mapping_radius_48htype1_48htype2}_{file_perfect_poscar_48n24_wo_cif}.pkl')

df_wo_weirdos_selectedcol = file_loc_important_cols[['geometry','path','subdir_new_system', 'toten [eV]']].sort_values("toten [eV]", ascending=True)[n_file_start:n_file_end].reset_index().copy()

In [ ]:
litype = 5
var_excel_file = "litype5"
max_mapping_radius_init = max_mapping_radius
max_mapping_radius_48htype2_init = max_mapping_radius_48htype2

dx1_48h_type1_init, dx2_48h_type1_init, dz_48h_type1_init, dx1_48h_type2_init, dx2_48h_type2_init, dz_48h_type2_init, dx_24g_init, dz1_24g_init, dz2_24g_init, dx1_48h_type3_init, dx2_48h_type3_init, dz_48h_type3_init, dx1_48h_type4_init, dx2_48h_type4_init, dz_48h_type4_init = get_dx_dz_init_litype(file_path_ori_ref_48n24, litype)

dx1_48h_type1, dx2_48h_type1, dz_48h_type1 = dx1_48h_type1_init, dx2_48h_type1_init, dz_48h_type1_init
dx1_48h_type2, dx2_48h_type2, dz_48h_type2 = dx1_48h_type2_init, dx2_48h_type2_init, dz_48h_type2_init
dx_24g, dz1_24g, dz2_24g = dx_24g_init, dz1_24g_init, dz2_24g_init
dx1_48h_type3, dx2_48h_type3, dz_48h_type3 = dx1_48h_type3_init, dx2_48h_type3_init, dz_48h_type3_init
dx1_48h_type4, dx2_48h_type4, dz_48h_type4 = dx1_48h_type4_init, dx2_48h_type4_init, dz_48h_type4_init

# idea: outer loop dz2 positive, then dz1 ositive and negative
# 2nd loop: dz2 negative

In [ ]:
delta_coor = 0.02

min_coor_dx2 = dx2_48h_type4 - 0.50 - delta_coor
max_coor_dx2 = dx2_48h_type4 + 0.50 + delta_coor

min_coor_dz = dz_48h_type4 - 0.50 - delta_coor
max_coor_dz = dz_48h_type4 + 0.50 + delta_coor

# delta_coor = 0.002

# min_coor_dx2 = dx2_48h_type4 - 0.005 - delta_coor
# max_coor_dx2 = dx2_48h_type4 + 0.005 + delta_coor

# min_coor_dz = dz_48h_type4 - 0.005 - delta_coor
# max_coor_dz = dz_48h_type4 + 0.005 + delta_coor

var_optitype = "varying_dx_dz"

In [ ]:
forward_dx1_init = dx1_48h_type4
forward_dx2_init = dx2_48h_type4
forward_dz_init = dz_48h_type4

backward_dx1 = dx1_48h_type4
backward_dx2 = dx2_48h_type4
backward_dz = dz_48h_type4

forward_dx1_negative = dx1_48h_type4 * -1
forward_dx2_negative = dx2_48h_type4 * -1
forward_dz_negative = dz_48h_type4 * -1

backward_dx1_negative = dx1_48h_type4 * -1
backward_dx2_negative = dx2_48h_type4 * -1
backward_dz_negative = dz_48h_type4 * -1

In [ ]:
col_dx1 = "dx1"
col_dx2 = "dx2"
col_dz = "dz"
col_sumweirdosLi = "sumweirdosLi"

dx_dz_sumweirdosLi_df = pd.DataFrame()
dx_dz_sumweirdosLi_df[col_dx1] = None
dx_dz_sumweirdosLi_df[col_dx2] = None
dx_dz_sumweirdosLi_df[col_dz] = None
dx_dz_sumweirdosLi_df[col_sumweirdosLi] = None

global idx_sumweirdosLi_df
idx_sumweirdosLi_df = 0


## all forward
forward_dx2 = dx2_48h_type4

while forward_dx2 <= max_coor_dx2:
    
    forward_dx1 = dx1_48h_type4
    forward_dz = dz_48h_type4
    
    while forward_dz <= max_coor_dz:
        sum_weirdos_Li = get_sum_weirdos_Li_var_wo_weirdo_litype4(forward_dx1, forward_dx2, forward_dz, max_mapping_radius, max_mapping_radius_48htype2, df_wo_weirdos_selectedcol, activate_radius, file_perfect_poscar_24, file_perfect_poscar_48n24, litype, var_optitype)
    
        print(f"dx1: {forward_dx1}, dx2: {forward_dx2}, dz: {forward_dz}, idx: {idx_sumweirdosLi_df}")

        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dx1] = forward_dx1
        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dx2] = forward_dx2
        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dz] = forward_dz
        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_sumweirdosLi] = sum_weirdos_Li

        forward_dz += delta_coor
        forward_dx1 -= delta_coor

        idx_sumweirdosLi_df += 1

    forward_dx2 += delta_coor


## inner loop backward
forward_dx2 = dx2_48h_type4

while forward_dx2 <= max_coor_dx2:
    
    forward_dx1 = dx1_48h_type4
    forward_dz = dz_48h_type4
    
    while forward_dz >= min_coor_dz:
        sum_weirdos_Li = get_sum_weirdos_Li_var_wo_weirdo_litype4(forward_dx1, forward_dx2, forward_dz, max_mapping_radius, max_mapping_radius_48htype2, df_wo_weirdos_selectedcol, activate_radius, file_perfect_poscar_24, file_perfect_poscar_48n24, litype, var_optitype)
    
        print(f"dx1: {forward_dx1}, dx2: {forward_dx2}, dz: {forward_dz}, idx: {idx_sumweirdosLi_df}")

        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dx1] = forward_dx1
        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dx2] = forward_dx2
        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dz] = forward_dz
        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_sumweirdosLi] = sum_weirdos_Li

        forward_dz -= delta_coor
        forward_dx1 += delta_coor

        idx_sumweirdosLi_df += 1

    forward_dx2 += delta_coor


## all backward
forward_dx2 = dx2_48h_type4

while forward_dx2 >= min_coor_dx2:
    
    forward_dx1 = dx1_48h_type4
    forward_dz = dz_48h_type4
    
    while forward_dz >= min_coor_dz:
        sum_weirdos_Li = get_sum_weirdos_Li_var_wo_weirdo_litype4(forward_dx1, forward_dx2, forward_dz, max_mapping_radius, max_mapping_radius_48htype2, df_wo_weirdos_selectedcol, activate_radius, file_perfect_poscar_24, file_perfect_poscar_48n24, litype, var_optitype)
    
        print(f"dx1: {forward_dx1}, dx2: {forward_dx2}, dz: {forward_dz}, idx: {idx_sumweirdosLi_df}")

        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dx1] = forward_dx1
        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dx2] = forward_dx2
        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dz] = forward_dz
        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_sumweirdosLi] = sum_weirdos_Li

        forward_dz -= delta_coor
        forward_dx1 += delta_coor

        idx_sumweirdosLi_df += 1

    forward_dx2 -= delta_coor


## outer backward
forward_dx2 = dx2_48h_type4

while forward_dx2 >= min_coor_dx2:
    
    forward_dx1 = dx1_48h_type4
    forward_dz = dz_48h_type4
    
    while forward_dz <= max_coor_dz:
        sum_weirdos_Li = get_sum_weirdos_Li_var_wo_weirdo_litype4(forward_dx1, forward_dx2, forward_dz, max_mapping_radius, max_mapping_radius_48htype2, df_wo_weirdos_selectedcol, activate_radius, file_perfect_poscar_24, file_perfect_poscar_48n24, litype, var_optitype)
    
        print(f"dx1: {forward_dx1}, dx2: {forward_dx2}, dz: {forward_dz}, idx: {idx_sumweirdosLi_df}")

        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dx1] = forward_dx1
        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dx2] = forward_dx2
        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dz] = forward_dz
        dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_sumweirdosLi] = sum_weirdos_Li

        forward_dz += delta_coor
        forward_dx1 -= delta_coor

        idx_sumweirdosLi_df += 1

    forward_dx2 -= delta_coor

In [ ]:
# def process_data(forward_dx2_start, forward_dx2_end, dx2_step, forward_dz_start, forward_dz_end, dz_step, delta_coor):
#     global idx_sumweirdosLi_df

#     forward_dx2 = forward_dx2_start

#     while forward_dx2 <= forward_dx2_end:
        
#         forward_dx1 = dx1_48h_type1
#         forward_dz = forward_dz_start
        
#         while forward_dz <= forward_dz_end:
#             sum_weirdos_Li = get_sum_weirdos_Li_var_wo_weirdo(forward_dx1, forward_dx2, forward_dz, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2, df_wo_weirdos_selectedcol, activate_radius, file_perfect_poscar_24, file_perfect_poscar_48n24, litype, var_optitype)
        
#             print(f"dx1: {forward_dx1}, dx2: {forward_dx2}, dz: {forward_dz}, idx: {idx_sumweirdosLi_df}")

#             dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dx1] = forward_dx1
#             dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dx2] = forward_dx2
#             dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_dz] = forward_dz
#             dx_dz_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_sumweirdosLi] = sum_weirdos_Li

#             forward_dz += delta_coor
#             forward_dx1 -= delta_coor

#             idx_sumweirdosLi_df += 1

#         forward_dx2 += dx2_step


# # All forward
# process_data(dx2_48h_type1, max_coor_dx2, delta_coor, dz_48h_type1, max_coor_dz, delta_coor, delta_coor)

# # Inner loop backward
# process_data(dx2_48h_type1, max_coor_dx2, delta_coor, dz_48h_type1, min_coor_dz, -delta_coor, delta_coor)

# # All backward
# process_data(max_coor_dx2, min_coor_dx2, -delta_coor, dz_48h_type1, min_coor_dz, -delta_coor, delta_coor)

# # Outer backward
# process_data(max_coor_dx2, min_coor_dx2, -delta_coor, dz_48h_type1, max_coor_dz, delta_coor, delta_coor)


In [ ]:
dx_dz_sumweirdosLi_df

In [ ]:
# dx_dz_sumweirdosLi_df.plot(x="radius_type1", y="sumweirdosLi")

In [ ]:
min_sumweirdos = int(dx_dz_sumweirdosLi_df["sumweirdosLi"].min())

In [ ]:
dx_dz_sumweirdosLi_df_noduplicates = dx_dz_sumweirdosLi_df.drop_duplicates(subset=['dx1', 'dx2', 'dz'])

In [ ]:
if activate_radius == 2:
    dx_dz_sumweirdosLi_df_noduplicates.to_excel(f'08_varying_dx_dz_{min_sumweirdos}_{max_mapping_radius}_{max_mapping_radius_48htype2}_{file_perfect_poscar_48n24_wo_cif}_{var_excel_file}.xlsx', index=False)
elif activate_radius == 1:
    dx_dz_sumweirdosLi_df_noduplicates.to_excel(f'08_varying_dx_dz_{min_sumweirdos}_{max_mapping_radius}_{file_perfect_poscar_48n24_wo_cif}_{var_excel_file}.xlsx', index=False)

In [ ]:
%matplotlib inline

# flights= sns.load_dataset('flights')

# flights.head()

matrix_map = dx_dz_sumweirdosLi_df_noduplicates.pivot_table(index='dx1', columns='dx2',values='sumweirdosLi')  

sns.heatmap(matrix_map)

In [ ]:
# df = pd.read_excel('08_varying_dx_dz_31_litype2.xlsx')

# selected_rows = df[(df['dx2'] >= 0.020) & (df['dx2'] <= 0.036)]

# matrix_map = selected_rows.pivot_table(index='dx1', columns='dx2',values='sumweirdosLi')  

# sns.heatmap(matrix_map)